# Artskart Analytical Database
### Data Engineering Capstone Project

#### Project Summary
In this project I'm doing a query optimized analytical database of Redlisted Species in Norway with Artskart. Further, enrich data with population growth and deforestation. Artskart is a Citizen science project for recording species on maps into a national and freely accessible database. Artskart is owned by The Norwegian Biodiversity Information Centre. You may read more about Artskart here: https://www.biodiversity.no/Pages/135580/About_Norwegian_Biodiversity_Information_Centre?Key=1435226530

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [24]:
# Import libraries
import findspark
findspark.init()

import pandas as pd
import configparser
from datetime import datetime
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

#### Scope 
I'm planning to do a query optimized analytical database of Redlisted Species in Norway. Further, enrich species data with population growth and deforestation. My primary datasource is Artskart, which is a Citizen science project for recording species on maps into a national and freely accessible database. Artskart is owned by The Norwegian Biodiversity Information Centre.
- Explore and clean data using Apache Spark (PySpark) and validate that all data is in the expected format
- Partition data into .csv files using PySpark and load them into a data lake in Amazon S3
- Load data into an analytical datatabase in Postgres using parallell ETL with Python and AWS boto3 library

#### Describe and Gather Data 
Artskart2Index is a Microsoft SQL server relational datatabase where data is broken down into multiple different tables in 3NF.
I'm using a pre-made view stored in the database's view folder to query redlisted species.
I've restored the database on my local machine using `.bak-file` containing all species observation until the 7. march 2021. 
* Restore Artskart2Index database using the .bak-file and a local instance of SQL Server Express
* Push down a query using Spark on a pre-processed database view fetching all redlisted species

In [ ]:
# Create SparkConf with native microsoft jdbc driver for sql-server
configure = SparkConf().setAppName("Artskart")\
                       .setMaster("local[*]")\
                       .set("spark.driver.extraClassPath","sqljdbc_9.2/enu/mssql-jdbc-9.2.1.jre8.jar")

In [ ]:
# Create SparkContext
sc = SparkContext(conf = configure)

In [ ]:
# Database credentials
jdbcHostname=<jdbcHostname>
jdbcPort=<jdbcPort>
jdbcDatabase=<jdbcDatabase>
jdbcUsername=<jdbcUsername>
jdbcPassword=<jdbcPassword>

In [ ]:
# JDBC connection URL
jdbcUrl = "jdbc:sqlserver://{0}:{1};Database={2};".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [ ]:
# Query Artskart database
pushdown_query = "(select * from View_ShapeExportRedlist) redlist_alias"

In [10]:
# Push down query 
# Read results into Spark dataframe
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)
display(df)

DataFrame[FID: int, ProxyId: string, NodeDatabaseID: int, InstitutionCode: string, CollectionCode: string, CatalogNumber: string, VitNavn: string, BasisOfRecord: string, Kingdom: string, Phylum: string, Class: string, Order: string, Family: string, Genus: string, Species: string, Subspecies: string, Author: string, IdentifiedBy: string, YearIdentified: int, MonthIdentified: int, DayIdentified: int, TypeStatus: string, CollectorNumber: string, FieldNumber: string, Collector: string, YearCollected: int, MonthCollected: int, DayCollected: int, ContinentOcean: string, Country: string, StateProvince: string, CountyOrg: string, Locality: string, CountyID: string, MunicipalityID: string, County: string, MuniName: string, Longitude: double, Latitude: double, CoordinatePrecision: int, BoundingBox: string, MinElevation: int, MaxElevation: int, MinDepth: string, MaxDepth: string, Sex: string, PreparationType: string, IndividualCount: string, PreviousCatalogNumber: string, RelationshipType: string

##### Following error might occur when connecting with native JDBC driver
`com.microsoft.sqlserver.jdbc.SQLServerException: The TCP/IP connection to the host HOSTNAME, port 1433 has failed.
Error: "Connection refused: connect. Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port.
Make sure that TCP connections to the port are not blocked by a firewall.`

Please read further here: https://kb.sos-berlin.com/pages/viewpage.action?pageId=17499564

In [17]:
# Set filename
file_name = "redlist.csv"

In [18]:
# Set output path to local working directory
out_path = os.path.abspath(file_name)

In [13]:
# Save a copy on my local working directory repartitioned by 8
df.repartition(8).write.save(out_path, format="csv", header=True) # row count: 4,384,290

In [20]:
# Read file back into RAM with Spark
redlist = spark.read.csv(out_path, header=True)

In [21]:
redlist.head()

Row(FID='41954328', ProxyId='urn:uuid:43688d2a-f87b-4031-9e20-c6e0ca85840a', NodeDatabaseID='1010', InstitutionCode='NOF', CollectionCode='so2-birds', CatalogNumber='24208745', VitNavn='Delichon urbicum', BasisOfRecord='humanobservation', Kingdom='Animalia', Phylum='Chordata', Class='Aves', Order='Passeriformes', Family='Hirundinidae', Genus='Delichon', Species='Delichon urbicum', Subspecies=None, Author='(Linnaeus, 1758)', IdentifiedBy=None, YearIdentified='0', MonthIdentified='0', DayIdentified='0', TypeStatus=None, CollectorNumber=None, FieldNumber=None, Collector='Frank Holmen', YearCollected='2020', MonthCollected='5', DayCollected='19', ContinentOcean=None, Country='Norway', StateProvince=None, CountyOrg=None, Locality='Møssevoll, Vessøyjordene, Grimstad, Ag', CountyID='42', MunicipalityID='4202', County='Agder', MuniName='Grimstad', Longitude='8.694183', Latitude='58.388015', CoordinatePrecision='116', BoundingBox=None, MinElevation=None, MaxElevation=None, MinDepth=None, MaxDep

In [ ]:
# Copy file to S3-bucket using AWS CLI
!aws s3 out_path s3a://<bucket-name> --recursive

![schematics](schematics.jpg "schematics")

### Step 2: Explore and Assess the Data

#### Explore the Data 
Identifying data quality issues, like missing values, duplicate data, etc.

#### Questions to ask: 
- How many observations of redlisted species do we have in our dataset
- Which redlisted specie were the most observed
- Which redlisted specie were the least observed
- How are the observations skeewed
- Do we have any duplicates rows
- Are there any corrupt rows

In [27]:
# How many observations of redlisted species do we have?
redlist.count()

4384290

In [28]:
# Do we have any duplicate rows?
duplicates = redlist.count() - redlist.distinct().count()
print(duplicates)

0


In [30]:
redlist.select(["County", "MuniName", "Country", "NorskNavn"]).take(5)

[Row(County='Agder', MuniName='Grimstad', Country='Norway', NorskNavn='taksvale'),
 Row(County='Trøndelag', MuniName='Levanger', Country='Norway', NorskNavn='stær'),
 Row(County='Trøndelag', MuniName='Ørland', Country='Norway', NorskNavn='storspove'),
 Row(County='Rogaland', MuniName='Stavanger', Country='Norway', NorskNavn='stær'),
 Row(County='Agder', MuniName='Farsund', Country='Norway', NorskNavn='sivspurv')]

#### Cleaning Steps
Document steps necessary to clean the data

### Fix schema
- make sure we have correct date formats
- prepare for parallell etl
    - include boto3 library
    - list bucket keys
    - read .csv table into redshift as is or prepartion tables in S3 as facts and dimensions
    - create postgres facts and dimension tables in redshift
    - insert data using copy command

In [ ]:
# Register the DataFrame as a SQL temporary view
redlist.createOrReplaceTempView("redlist")

In [22]:
# Which redlisted specieS are the most observed ones?
species_observed = spark.sql("""
    SELECT SPECIES,
           COUNT(*) AS CNT
    FROM REDLIST
    GROUP BY SPECIES
    ORDER BY CNT DESC
    limit 5
""").show()

+--------------------+------+
|             SPECIES|   CNT|
+--------------------+------+
|         Larus canus|364333|
|Somateria mollissima|300448|
|    Sturnus vulgaris|266324|
| Emberiza citrinella|253517|
|Chroicocephalus r...|189492|
+--------------------+------+



In [21]:
# Count distinct redlisted species observed
num_of_species = spark.sql("""
    SELECT COUNT(DISTINCT SPECIES) AS NUM_OF_SPECIES
    FROM REDLIST
""").show()

+--------------+
|NUM_OF_SPECIES|
+--------------+
|          4075|
+--------------+



In [27]:
# Check how the observations are distributed on "MonthIdentified"
dist_identified = spark.sql("""

    SELECT MONTHIDENTIFIED,
           COUNT(*) AS CNT
    FROM REDLIST
    GROUP BY MONTHIDENTIFIED
    ORDER BY MONTHIDENTIFIED ASC
    
""").show()

+---------------+-------+
|MONTHIDENTIFIED|    CNT|
+---------------+-------+
|              0|4215033|
|              1|  23978|
|             10|  15870|
|             11|   5216|
|             12|   3809|
|              2|   3400|
|              3|   2485|
|              4|   4090|
|              5|   9104|
|              6|  20445|
|              7|  23163|
|              8|  29187|
|              9|  28510|
+---------------+-------+



In [33]:
# Check how the observations are distributed on "MonthCollected"
dist_identified = spark.sql("""
    -- Check how data is distributed on "MonthCollected"
    SELECT MONTHCOLLECTED,
           COUNT(*) AS CNT
    FROM REDLIST
    GROUP BY MONTHCOLLECTED
    ORDER BY MONTHCOLLECTED ASC
    
""").show()

+--------------+------+
|MONTHCOLLECTED|   CNT|
+--------------+------+
|             1|271527|
|             2|182156|
|             3|307859|
|             4|544599|
|             5|710578|
|             6|532953|
|             7|408897|
|             8|460916|
|             9|347744|
|            10|302214|
|            11|182210|
|            12|132637|
+--------------+------+

